# A0149963M
## Assignment 4
## YSC2229 - Introductory Data Structures and Algorithms

### Solution for Problem 1 - Runway Scheduling System

**1(a) Store an approved/valid request in O(log n), where the stored information includes: aircraft ID (with format "AA123456") and take-off time**

Before starting on this question, I'd like to define the following basic objects first. Firstly, i define the constants Red and Black which constitute the basis of Red-Black Trees. Secondly, I define the class RBNode which represents a Node in the Red-Black Tree. This stores the following attributes: the time (in 24h format), its ID number, its parent (self.p), left and right child, colour (red by default) and info, which will be covered later when we talk about augmenting the Red-Black Tree in order to output requests in **O(log n)**. In addition, I will store the time object, which is needed to store the airplane times. 

In [1]:
Red = 'Red'
Black = 'Black'

class RBNode():
    def __init__(self, id_no = None, time = None, info = None):
        self.time = time
        self.id_no = id_no
        self.p = None
        self.left = None
        self.right = None
        self.col = Red 
        self.info = info

class Time():
    def __init__(self, h, minute, sec):
        """
        Stores time in hours, minutes
        and seconds. 
        """
        self.check_validity(h, minute, sec)
        self.h = h
        self.minute = minute
        self.sec = sec 
    
    def check_validity(self, h, minute, sec):
        """
        Utility function to check if time inputted
        is a valid time to be registered. 
        """
        if type(h) != int or type(minute) != int or type(sec) != int:
            raise TypeError("Please input Integer values only!")
        
        if h < 0 or h > 23:
            raise ValueError("Please Input integer hour value between 0 and 23 inclusive.")
        
        if minute < 0 or minute > 59:
            raise ValueError("Please Input integer minute value between 0 and 59 inclusive.")
    
        if sec < 0 or sec > 59:
            raise ValueError("Please Input integer second value between 0 and 59 inclusive.")
    
    def in_sec(self):
        """
        Converts the time into seconds. This helps to directly compare 
        2 times and see which one is later and which is earlier. 
        """
        n_sec = self.h * 3600 + self.minute * 60 + self.sec
        return n_sec
    
    def fix_time(self, unit):
        """
        Utility function to help express time in terms of hh-mm-ss format
        """
        if unit < 10:
            return "0" + str(unit)
        else:
            return str(unit)
    
    def in_str(self):
        """
        Pretty print to express term in hh-mm-ss format 
        """
        str_time = self.fix_time(self.h) + ":" + self.fix_time(self.minute) + ":" + self.fix_time(self.sec)
        return str_time
    
    

Next, I will be doing brief tests of the Time object and demonstrate what it is supposed to do that I have defined above. In the next isolated square, I have also tested the **check_validity** auxillary function that I have defined for time and created some illegal expressions that I have commented out. feel free to uncomment it if you'd like to test it.

In [2]:
# testing time initialization
# input in hours, then minutes, then seconds
t = Time(22, 34, 10)

# gives 81250 seconds, which is correct
t.in_sec()

# pretty print format
print("24h format time of Time(22, 34, 10):" ,t.in_str())

# Now demonstrating fix_time for units < 10
t = Time(8, 5, 3)
print("24h format time of Time(8, 5, 3):" ,t.in_str())

24h format time of Time(22, 34, 10): 22:34:10
24h format time of Time(8, 5, 3): 08:05:03


In [3]:
# t = Time("", 3, 1)
# t = Time(2, 2.3, 1)
# t = Time(25, 0, 1)
# t = Time(23, -1, 3)
# t = Time(20, 1, 65)

With this set in stone, it is time to define the Red-Black Tree proper. The code of this follows the lecture notes very closely, and it contains all necessary operations such as **left_rotate**, **right_rotate**, **fix_insert**, **insert**, etc. However, for this question, it is mentioned that a request is stored/approved if is is not within **k** minutes from the previous and the next requests that are already stored in the system. 

Therefore, we have to define a few additional functions in order to address this requirement. In particular, there must be an auxillary function that checks the parent, left and right node to verify whether it is within **k** minutes from the previous and the next requests that are already stored in the system. In addition, we need to define a new attribute delay that becomes the basis of comparison when checking for this requirement. 

Other than that, the Red Black Tree has been calibrated in order to search for Time With these requirements set in stone, let's go ahead and create the Red-Black Tree data structure:

In [4]:
class RedBlackTree():
    def __init__(self, delay):
        """
        NIL node is defined as per normal, but
        delay is converted from minutes to seconds.
        This makes for easy comparison with the 
        in_sec operation defined earlier in Time().
        """
        self.delay = 60 * delay
        self.NIL = RBNode()
        self.NIL.col = Black
        self.NIL.left = None
        self.NIL.right = None
        self.root = self.NIL
    
    def left_rotate(self, x):
        """
        Conventional rotation operation taken
        from the module textbook
        """
        y = x.right
        x.right = y.left
        if y.left != self.NIL:
            y.left.p = x
        
        y.p = x.p
        if x.p == self.NIL:
            self.root = y
        elif x == x.p.left:
            x.p.left = y
        else:
            x.p.right = y
        y.left = x
        x.p = y
    
    def right_rotate(self, x):
        """
        Same as left_rotate, just mirrored
        """
        y = x.left
        x.left = y.right
        if y.right != self.NIL:
            y.right.p = x
        
        y.p = x.p
        if x.p == self.NIL:
            self.root = y
        elif x == x.p.right:
            x.p.right = y
        else: 
            x.p.left = y
        y.right = x
        x.p = y
    
        
    def fix_insert(self, z):
        """
        Adjustment procedure to maintain
        Red-Black property! 
        """
        while z.p.col == Red:
            if z.p == z.p.p.left:
                y = z.p.p.right
                if y.col == Red:
                    z.p.col = Black
                    y.col = Black
                    z.p.p.col = Red
                    z = z.p.p
                else:
                    if z == z.p.right:
                        z = z.p
                        self.left_rotate(z)
                    z.p.col = Black
                    z.p.p.col = Red
                    self.right_rotate(z.p.p)
            else:
                y = z.p.p.left
                if y.col == Red:
                    z.p.col = Black
                    y.col = Black
                    z.p.p.col = Red
                    z = z.p.p
                else:
                    if z == z.p.left:
                        z = z.p
                        self.right_rotate(z)
                    z.p.col = Black
                    z.p.p.col = Red
                    self.left_rotate(z.p.p)
                    
        self.root.col = Black
      
        
    def insert(self, id_key, hh_key, mm_key, ss_key):
        """
        Insert operation. The only modification here
        is that instead of storing an integer, a Time
        object is stored instead. 
        """
        time_key = Time(hh_key, mm_key, ss_key)
        z = RBNode(id_key, time_key)
        y = self.NIL
        x = self.root
        
        while x != self.NIL:
            if abs(x.time.in_sec() - z.time.in_sec()) < self.delay:
                print("Request for", z.id_no, "for takeoff not approved\n")
                return
            y = x
            if z.time.in_sec() < x.time.in_sec():
                x = x.left
            else: 
                x = x.right
            
        z.p = y
        if y == self.NIL:
            self.root = z
        elif z.time.in_sec() < y.time.in_sec():
            y.left = z
        else: 
            y.right = z
        
        z.left = self.NIL
        z.right = self.NIL
        z.col = Red
        
        self.fix_insert(z)
        
        
    def find_min(self):
        """
        Function to find minimum value of Red-Black Tree
        by traversing all the way to the left-most node. 
        """
        node = self.root
        while node.left != self.NIL:
            node = node.left
        print("Minimum time is:", node.time.in_str())
        print("Corresponding ID number is:", node.id_no, "\n")
    
    def find_max(self):
        """
        Function to find maximum value of Red-Black Tree
        by traversing all the way to the left-most node. 
        """
        node = self.root
        while node.right != self.NIL:
            node = node.right
        print("Maximum time is:", node.time.in_str())
        print("Corresponding ID number is:", node.id_no, "\n")            
        
    def print_node(self, x):
        """
        Auxillary function to pretty print node values. 
        """
        print("ID:", x.id_no, ", Time:", x.time.in_str())
    
    def inorder(self, node):
        """
        In order traversal of the node that 
        also prints values on the way.
        """
        if node != self.NIL:
            self.inorder(node.left)
            self.print_node(node)
            self.inorder(node.right)
    
    
    def populate_info(self, node):
        """
        Populates the node with information about
        its sub-tree. Very useful for the search
        operation later on.
        """
        if node is None or node.time == None:
            return []
        left = self.populate_info(node.left)
        right = self.populate_info(node.right)
        node.info = left + [(node.id_no, node.time)] + right
        return node.info

When inserting data into this Red-Black Tree, please take note on how to add the node. The insert function takes 4 arguments, in this order: **tree.insert(ID_No, hh, mm, ss)**. Subsequently, **hh, mm, ss** is converted to a time object and added to the tree. Some examples on how to do so are shown below, together with informative print messages. 

The below function shows the storing of approved/valid requests. The last request, UC213141 is not approved since it is within 3 minutes of the request TY856120. In addition, my function implementation here stores the earliest and latest take-off time of the plane and performs an inorder traversal of the plane from the earliest to latest time.

In [5]:
t = RedBlackTree(3) # min delay is 3min
t.insert("AR190237", 13, 34, 59) # 13:34:59
t.insert("AS123984", 21, 12, 0)  # 21:12:00
t.insert("AE123973", 19, 7, 12)  # 19:07:00 
t.insert("AR812731", 15, 12, 9)  # 15:12:09
t.insert("GC123124", 17, 18, 40) # 17:18:40
t.insert("TY856120", 23, 11, 3)  # 23:11:03

 
t.find_min()
t.find_max()
t.inorder(t.root)

Minimum time is: 13:34:59
Corresponding ID number is: AR190237 

Maximum time is: 23:11:03
Corresponding ID number is: TY856120 

ID: AR190237 , Time: 13:34:59
ID: AR812731 , Time: 15:12:09
ID: GC123124 , Time: 17:18:40
ID: AE123973 , Time: 19:07:12
ID: AS123984 , Time: 21:12:00
ID: TY856120 , Time: 23:11:03


**(b) Given start and end times, output requests (aircraft IDs and the corresponding take-off times) in O(log n)**

In order to do this question, we need a binary search algorithm that operates in **O(log n)** time. However, this is not easy since we need to output a range. Therefore, we best we can do is to populate the all subtrees with information about itself as a flattened list so this can be easily called at O(1) time. In addition, we need to be a little bit creative with our algorithm. Here is an intuition behind this algorithm: 

**out_request** - it takes in a **red-black tree, h(starting time), min(starting time), s(starting time), h(end time), min(end time), s(end time)** in order. There are 7 arguments here, so please input cautiously. Firstly, this function defines a start and end Time() object. Afterwards, it traverses the tree from its root node. Trying to find a node with time within the range while it has not encountered a NIL node. If this is possible, then the while loop breaks prematurely. 

If the time is earlier than the range stated, the node traverses to the left. If the time is later than the range stated, the node traverses to the right. This repeats until either the node is a NIL node or it terminates after finding something in range. If the node is a NIL node, then there are no times in the red black tree within the range, if which it returns None. However, the fun begins when it starts to mop up information to find the left and right bound. The right child will be inputted to **print_req_right** while the left child will be inputted to **print_req_left**, 

In **print_req_right**, the tree tries to traverse all the way to the right until it finds the first timing later than the latest time in the time range. However, as it tries to find it, it prints information about 
 - itself, since it is still within the time range
 - left subtree augmented information, since all these timings are earlier than this parent node and therefore within the range.


**print_req_left** does something very similar the tree tries to traverse all the way to the left until it finds the first timing earlier than the earliest time in the time range. However, as it tries to find it, it prints information about 
 - itself, since it is still within the time range
 - right subtree augmented information, since all these timings are later than this parent node and therefore within the range.
 
This explains the search algorithm that I've defined for this question.

In [6]:
def print_req_right(node, end):
    """
    end - latest time of range
    node - current node traversed
    
    Algorithm to try and find the right bound of the time range.
    Starts from the left child of the first node found in range.
    """
    # if node is nil, don't bother finding time
    if node.time == None:
        return
    
    # while the nodes are still in range (earlier than latest time)
    while node.time.in_sec() < end.in_sec():
        print("ID:", node.id_no + ", Time", node.time.in_str())
        if node.left.info != None:
            # print augmented information of all subtrees within range
            for (id_no, time) in node.left.info:
                print("ID:", id_no + ", Time", time.in_str())
                
        # break if encountering nil node, if not continue
        if node.right.time == None:
            break
        else: 
            node = node.right
        
def print_req_left(node, start):
    """
    start - earliest time of range
    node - current node traversed
    
    Algorithm to try and find the left bound of the time range.
    Starts from the right child of the first node found in range.
    """
    # if node is nil, don't bother finding time
    if node.time == None:
        return
    # while the nodes are still in range (later than earliest time)
    while node.time.in_sec() > start.in_sec():
        print("ID:", node.id_no + ", Time", node.time.in_str())
        if node.right.info != None:
            # print augmented information of subtrees within range
            for (id_no, time) in node.right.info:
                print("ID:", id_no + ", Time", time.in_str())
            
        # break if encountering nil node, if not continue
        if node.left.time == None:
            break
        else:    
            node = node.left

def out_request(RBt, start_h, start_m, start_s, end_h, end_m, end_s):
    """
    Algorithm to output requests in O(log(n)). 
    Explained in greater depth above. 
    """
    RBt.populate_info(RBt.root)
    end = Time(end_h, end_m, end_s)
    start = Time(start_h, start_m, start_s)
    in_range = RBt.root
    
    # traverses to find the first node that is within range 
    while in_range.time != None:
        if start.in_sec() <= in_range.time.in_sec() and in_range.time.in_sec() <= end.in_sec():
            break
        # next 2 conditions where in_range is out of range
        elif in_range.time.in_sec() < start.in_sec():
            in_range = in_range.right
        elif in_range.time.in_sec() > end.in_sec():
            in_range = in_range.left
        
    # if no node in range
    if in_range == RBt.NIL:
        print("No available timings in range. returning empty list")
        return
    
    # print subtree node first before continuing with left and right child
    print("ID:", in_range.id_no + ", Time", in_range.time.in_str())
    
    # now, in_range is truly in range. 
    in_range_too = in_range
    print_req_right(in_range.right, end)
    print_req_left(in_range_too.left, start)
    

    
t = RedBlackTree(3) # min delay is 3min
t.insert("AR190237", 1, 34, 59) 
t.insert("AS123984", 2, 12, 0)  
t.insert("AR190237", 3, 7, 12)  
t.insert("AR812731", 4, 12, 9)  
t.insert("AR190237", 5, 18, 40) 
t.insert("TY856120", 6, 11, 3)  
t.insert("AR190237", 7, 13, 3)  
t.insert("AR190237", 8, 13, 3)
t.insert("UI827313", 9, 13, 3)    
    
# in range test, and it gets all expected times. 
out_request(t, 5, 4, 1, 8, 51, 1)

ID: TY856120, Time 06:11:03
ID: AR190237, Time 08:13:03
ID: AR190237, Time 07:13:03
ID: AR190237, Time 05:18:40


In [7]:
t = RedBlackTree(3) # min delay is 3min
t.insert("AR190237", 13, 34, 59) # 13:34:59
t.insert("AS123984", 21, 12, 0)  # 21:12:00
t.insert("AE123973", 19, 7, 12)  # 19:07:00 
t.insert("AR812731", 15, 12, 9)  # 15:12:09
t.insert("GC123124", 17, 18, 40) # 17:18:40
t.insert("TY856120", 23, 11, 3)  # 23:11:03

# out of range test for 01:01:01 - 02:02:02.
out_request(t, 1, 1, 1, 2, 2, 2)

No available timings in range. returning empty list


**(c) Search requests based on an aircraft ID, where the output should be the aircraft ID and the
take-off times. Note, an aircraft ID might be stored multiple times.**

For this question, we need to traverse the tree and search requests based on aircraft ID. However, aircraft ID is not what we use to sort our Red-Black Tree. Therefore, it is necessary for us to search the whole tree in **O(n)** time and check what node has an aircraft ID that matches what we are searching for. We test this implementation below, and realize that it gives us a list of all the aircraft IDs that match "AR190237", which is what is needed of the question.

The output is a list of the flight number and takeoff time, which is what is demanded by the question.

In [8]:
t = RedBlackTree(3) # min delay is 3min
t.insert("AR190237", 1, 34, 59) 
t.insert("AS123984", 2, 12, 0)  
t.insert("AR190237", 3, 7, 12)  
t.insert("AR812731", 4, 12, 9)  
t.insert("AR190237", 5, 18, 40) 
t.insert("TY856120", 6, 11, 3)  
t.insert("AR190237", 7, 13, 3)  
t.insert("AR190237", 8, 13, 3)
t.insert("UI827313", 9, 13, 3)


def search_id(node, key, acc = []):
    """
    Searches planes with matching ID numbers.
    DFS, left to right traversal means that 
    the list of matching ID planes are also
    returned in increasing order (sorted).
    """
    if node.id_no == None:
        return 
    if node.id_no == key:
        acc.append((node.id_no, node.time.in_str()))
    search_id(node.left, key, acc)
    search_id(node.right, key, acc)
    return acc
        
acc = search_id(t.root, "AR190237")
print(acc)

[('AR190237', '01:34:59'), ('AR190237', '03:07:12'), ('AR190237', '05:18:40'), ('AR190237', '08:13:03'), ('AR190237', '07:13:03')]


Now that we are done with testing the function, 

### Solution for Problem 2 - Belief Propogation

For this question, I have to solve the belief propogation problem. In this problem, I have to solve the Belief Propogation Problem using Dynamic programming. I start from the first part:

**(a) Argue or justify that the problem can be solved using dynamic programming.**

This problem can be solved using dynamic programming. This is because of the overlapping substructures of the subproblems. In other words, subproblems share subsubproblems. If dynamic programming is not used, then the algorithm becomes inefficient because we have to compute the same subsubproblem multiple times. However, dynamic programing allows us to solve these subsubproblems just once and store its answer in a table, which can be queried, making our algorithm much more efficient. 

For a concrete example, consider the undirected graph consisting of the following:
 - Vertices consist of 1, 2, 3
 - Edge consists of (1, 2) and (1, 2)
Say, you'd like to calculate the eventual beliefs of all the vertices in this graph and you start from Node a. Following the definition defined in the assignment brief, then you have to calculate the following:

$$
b(x_{1} = T) = \phi(x_{1} = T) \times m_{12}(x_{1} = T) 
$$
$$
b(x_{1} = F) = \phi(x_{1} = F) \times m_{12}(x_{1} = F)
$$

Where $m_{12}$ represents the message from Vertice 2 to Vertice 1 (order swapped a little from the assignment brief). However, we also note that in the recursive message step, Vertice 2 has neighbours 1 and 3. In other words, we need to calculate $m_{23}$. In the recursive step defined in the lecture notes, 

$$
m_{ij} = \sum\phi(x_{i})\psi(x_{i}, x_{j}) \prod_{k \in N_{j} \setminus i} m_{jk}(x_{i})
$$

Observe that $\sum\phi(x_{i})\psi(x_{i}, x_{j})$ is a constant time step, but in the case of vertice 2, using this formula, we need to find $m_{23}(x_{2} = T)$ and $m_{23}(x_{2} = F)$. However, this step is repeated twice because $m_{12}(x_{1} = T)$ computes it once, then $m_{12}(x_{1} = F)$ computes it again. This repetition is inefficient and we can memoize it so that the time complexity is drastically reduced.

Before moving on to the next part, allow me to define some of the basic classes and objects we need to solve a question. In this code block, we have the class **Influence**, which stores the respective $\psi$ values between edges. This allows users to either set a value for $\psi$, or let the value be randomized. In this implementation,
- self.AND refers to the interaction coefficient when both nodes are F or T
- self.XOR refers to the interaction coefficient when the edge has nodes with either F-T or T-F. 

The same can be said for the Nodes. it has two attributes, **self.T** and **self.F** which represents the belief that something is true and false respectively. As per the Influence, one can either choose to set the coefficient themselves or let the computer set a randomized one. In addition, it contains the attribute **label** which helps when finalizing when confirming the result of which belief it should belong to: T or F. 

Lastly, **self.link** is a dictionary attribute containing all immediate neighbours it is connected to as keys. The values would then be the Influence, the $\psi$ that is between them. This helps later on significantly in the algorithm.

In [9]:
import random

class Influence:
    
    def __init__(self, coef = None):
        """
        XOR for F-T, T-F between nodes,
        AND for F-F, T-T between nodes.
        """
        self.coef = coef
        self.AND = random.random()
        self.XOR = 1 - self.AND
        self.set_param()
        
    def set_param(self):
        """
        Auxillary function to set a defined value for psis. 
        Sets AND value if user inputs, then sets the XOR
        belief as 1 - self.AND
        """
        if self.coef and (self.coef >= 0 and self.coef <= 1):
            self.AND = self.coef
            self.XOR = 1 - self.coef
        elif not self.coef:
            return
        else:
            print("Please input a probability between 0 and 1. Randomized XOR, AND used instead")
            return

class Node:
    def __init__(self, name, coef = None):
        self.coef = coef 
        self.name = name
        self.T = random.random()
        self.F = 1 - self.T
        self.label = None # not sure whether needed 
        self.link = {}
        self.set_param()
        
    def set_param(self):
        """
        Auxillary function to set a defined value for T/F. 
        Sets T value if user inputs, then sets the F
        belief as 1 - self.T.
        """
        if self.coef and (self.coef >= 0 and self.coef <= 1):
            self.T = self.coef
            self.F = 1 - self.coef
        elif not self.coef:
            return
        else:
            print("Please input a probability between 0 and 1. Randomized T and F used instead")
            return

However, these preliminary classes are supposed to be used in conjunction with a **Graph** class which is defined below. In it, we formalize the graph, we witness an example of calling the Influence and Node class. Since the Influence class is essentially an edge with extra information about the $\psi$, we need the Graph class as a binding mechanism for the **Node** and **Influence** class to work properly together. 

Here, we have the Graph class that contains the graph, **self.G** and the memo, **self.memo**. The latter is important for the dynamic programming section of the algorithm. Below shows the implementation of the Graph:

In [10]:
class Graph:
    def __init__(self):
        self.G = {}
        self.memo = {}
    
    def add_node(self, name, coef = None):
        """
        Auxillary function to add node to Graph.
        Works in the same way conventionally. 
        However, if a node has already been added,
        it cannot be overwritten (Will mess up 
        the Graph). Please redefine your graph. 
        """
        n = Node(name, coef)
        if n.name in self.G:
            print("Node already in Graph. Aborting... ")
            return
        else:
            self.G[n.name] = n
            return
    
    def add_edge(self, n1, n2, coef = None):
        """
        Auxillary function to add edge and Influence
        to Graph. However, If the 2 nodes are not in 
        the graph, the code will stop, prompting the 
        user to add it again. 
        """
        if n1 in self.G and n2 in self.G:
            i = Influence(coef)
            self.G[n1].link[n2] = i
            self.G[n2].link[n1] = i
        else:
            print("2 nodes not found in graph. Aborting... ")
            return 
            
    def print_graph(self):
        """
        This function pretty prints the whole graph, traversing
        and detailing all nodes with its immediate connections.
        Useful if one needs visualization.
        """
        for node in self.G:
            print("Now looking at Node", self.G[node].name) 
            print("T:", self.G[node].T, ", F:", self.G[node].F)
            print("Neighbouring edges: ")
            for edge, val in self.G[node].link.items():
                print("Connection between Node", node, "and", edge + ":")
                print("XOR val:", val.XOR) 
                print("AND val:", val.AND, "\n")
            

The below shows a few examples of how to add nodes and edges to the graph. The procedure is simple, use **add_node** if you need to add a Vertice to the graph and supply the label (preferably string). If you want a random T and F label, do not input the second argument. If you want a customized T and F label, please input a value between 0 and 1 (or else my code will randomize it instead). This will set the T value to your input and F value to 1 - (your input).

For the **add_edge** function to add an edge between 2 graphs, please add the 2 edges you'd like (that already exist in the graph. If not, my function will remind you to do so). If you'd like a customized $\psi$, please input a value between 0 and 1 on the third argument. This will set the AND value (T-T, F-F) to the value you inputted, and the XOR value to be 1 - AND. If not, it'll be randomized. The below shows such an example:

In [11]:
q = Graph()
q.add_node("a")           # random
q.add_node("b", 0.4)      # user determined
q.add_node("c")           # random 
q.add_edge("a", "b", 0.3) # user determined AND value
q.add_edge("b", "c")      # randomized psis

q.print_graph()

Now looking at Node a
T: 0.43179811110738864 , F: 0.5682018888926114
Neighbouring edges: 
Connection between Node a and b:
XOR val: 0.7
AND val: 0.3 

Now looking at Node b
T: 0.4 , F: 0.6
Neighbouring edges: 
Connection between Node b and a:
XOR val: 0.7
AND val: 0.3 

Connection between Node b and c:
XOR val: 0.07012705623704063
AND val: 0.9298729437629594 

Now looking at Node c
T: 0.3635316223502324 , F: 0.6364683776497676
Neighbouring edges: 
Connection between Node c and b:
XOR val: 0.07012705623704063
AND val: 0.9298729437629594 



Now that we are done with the implementation we are ready to take the next part: 

**(b) Write pseudocode for the dynamic programming solution to compute the belief of every
person’s eventual belief in the system.** 

To do this question, we need to note that we need to memoize the output of the messages to avoid recalculation. To do that, we pass the graph by reference so that the message values can be memoized by the **self.memo** attribute that we have defined just now that contains the memoized messages to avoid unnecessary recalculation. In this case, the below shows my pseudocode for calculating the eventual beliefs of all the vertices in the graph:

**Pseudocode for the auxillary function to find message with DP Methodology**

```
message(target, source, belief, graph):
    if belief == "T":
        val = source.link[target.name].AND * source.T + source.link[target.name].XOR * source.F
        for node in source.link:
        
            if node.name != target.name:
            
                if (target.name, source,name, "T) in graph.memo:
                    val  *= graph.memo[(target.name, source.name, "T")]
                else:
                    val *= message(source, node, "T", graph)
                    graph.memo[(target.name, source,name, "T)] = val
                    
                if (target.name, source,name, "F") in graph.memo:
                    val  *= graph.memo[(target.name, source.name, "F")]
                else:
                    val *= message(source, node, "F", graph)
                    graph.memo[(target.name, source,name, "F)] = val
                    
                    
    else:
        val = source.link[target.name].XOR * source.T + source.link[target.name].AND * source.F
        for node in source.link:
        
            if node.name != target.name:
            
                if (target.name, source,name, "F") in graph.memo:
                    val  *= graph.memo[(target.name, source.name, "F")]
                else:
                    val *= message(source, node, "F", graph)
                    graph.memo[(target.name, source,name, "F")] = val
                    
                if (target.name, source,name, "T") in graph.memo:
                    val  *= graph.memo[(target.name, source.name, "T")]
                else:
                    val *= message(source, node, "T", graph)
                    graph.memo[(target.name, source,name, "T")] = val
                    
    return val
``` 
For this algorithm, the broad methodology can be summarized as such:
1. Calculate the sum of the messages first: $\sum\phi(x_{i})\psi(x_{i}, x_{j})$


2. When recursing the message function, determine whether the neighbouring node has been visited. If yes, then don't recurse. If no, then recurse. 

    a. Determine whether the belief is True or False first
    
    b. Perform step 2 on each branch 
    
    c. When multiplying the message using $\prod_{k \in N_{j} \setminus i} m_{jk}(x_{i})$, for both the False and True case, check if the value has already been calculated.
    
    d. If yes, then take the value from the memo, If no, then calculate it manually using the formula defined in the assignment brief and store it afterwards when computed.

With this implementation, we can efficiently calculate the message of each source node and target node. This DP methodology largely reduces the amount of time required to compute the eventual belief, as spoken earlier. Now that we are done with this, let's move to the pseudocode to calculate belief.

**Pseudocode for the main function to find belief**

```
find_belief(graph):
    for node in graph:
        belief_true = node.T
        belief_false = node.F
        for neighbour in node.neighbours:
            source = neighbour
            belief_true *= message(node, source, "T", graph)
            belief_false *= message(node, source, "F", graph)
        
        if belief_true > belief_false:
            node.label = "T"
        else:
            node.label = "F"
    return
```     

This implementation is the manifestation of the formula $b(x_{i}) = \phi(x_{i}) \prod_{j = N_{i}} m_{ji}(x_{i})$. Nothing too fancy here, except that in the main code, it is necessary for to add informative print messages in order for the user to properly ascertain the eventual beliefs of each node. 

Now that we are done with this, we can move on to the next part: 


**(c) Write code in python based on your pseudocode, where the graph traversal must be depth first.** 

The implementation is practically the same as whatever we have done before. The trick here is to add informative print messages for the user to understand what are the eventual beliefs of each node. Since I have already touched on the pseudocode earlier, allow me to show the implementation:

In [12]:
def message(t, s, b, graph):
    """
    t - target (The node where the message is propogating to)
    s - source (The node where the message is propogating from)
    b - belief (True or False, "T" or "F" for computing belief)
    graph - Graph (pass by reference for memoization)
    """
    if b == "T":
        val = s.link[t.name].AND * s.T + s.link[t.name].XOR * s.F
        for n in s.link:
            if n != t.name:
                
                if (t.name, s.name, "T") in graph.memo:
                    val *= graph.memo[(t.name, s.name, "T")]
                else:
                    val *= message(s, graph.G[n], "T", graph)
                    graph.memo[(t.name, s.name, "T")] = val 
                    
                if (t.name, s.name, "F") in graph.memo:
                    val *= graph.memo[(t.name, s.name, "F")]
                else:
                    val *= message(s, graph.G[n], "F", graph) 
                    graph.memo[(t.name, s.name, "F")] = val 
    else:
        val = s.link[t.name].XOR * s.T + s.link[t.name].AND * s.F
        for n in s.link:
            if n != t.name:
                
                if (t.name, s.name, "F") in graph.memo:
                    val *= graph.memo[(t.name, s.name, "F")]
                else:
                    val *= message(s, graph.G[n], "F", graph)
                    
                if (t.name, s.name, "T") in graph.memo: 
                    val *= graph.memo[(t.name, s.name, "T")]
                else: 
                    val *= message(s, graph.G[n], "T", graph)
                    graph.memo[(t.name, s.name, "T")] = val
    return val 


def find_belief(graph):
    """
    Main function to find out the belief of each node 
    in the graph. This has an auxillary function
    message which is recursive and employs dynamic programming. 
    """
    for (label, node) in graph.G.items():
        b_true = node.T
        b_false = node.F
        for (key, _) in node.link.items():
            source = graph.G[key]
            b_true *= message(node, source, "T", graph)
            b_false *= message(node, source, "F", graph)
            
        # print messages to indicate eventual belief 
        print("For Node", label)
        print("Belief, T:", b_true)
        print("Belief, F:", b_false)
        node.label = "T" if b_true > b_false else "F"
        print("Eventual belief:", node.label, "\n")
    return 


**(d) Provide at least one example of the input and the output.**

The next 2 codeblocks show the fucntion successfully passing the tests that are shown in lecture note 11:

In [13]:
q = Graph()
q.add_node("a", 0.7)
q.add_node("b", 0.4)
q.add_edge("a", "b", 0.8)
find_belief(q)

For Node a
Belief, T: 0.308
Belief, F: 0.168
Eventual belief: T 

For Node b
Belief, T: 0.24799999999999997
Belief, F: 0.22799999999999998
Eventual belief: T 



In [14]:
q = Graph()
q.add_node("1", 0.7)
q.add_node("2", 0.4)
q.add_edge("1", "2", 0.1)
find_belief(q)

For Node 1
Belief, T: 0.406
Belief, F: 0.12600000000000003
Eventual belief: T 

For Node 2
Belief, T: 0.13600000000000004
Belief, F: 0.396
Eventual belief: F 



In order to prove that our implementation is efficient, we need to test this implementation against larger graphs. In this case, let's try to define a test that tests our graph against large inputs and see whether it diverges or takes too long to solve. Below shows a straight graph with roughly 1600 nodes. Let's see whether our algorithm diverges or not:

In [15]:
def stress_belief_mini():
    q = Graph()
    node_list = []
    for i in range(49, 51):
        for j in range(49, 51):
            label = chr(i) + chr(j)
            node_list.append(label)
            q.add_node(label)
    
    for i in range(len(node_list) - 1):
        q.add_edge(node_list[i], node_list[i + 1])
    
    find_belief(q)


def stress_belief():
    q = Graph()
    node_list = []
    for i in range(50, 91):
        for j in range(50, 91):
            label = chr(i) + chr(j)
            node_list.append(label)
            q.add_node(label)
    
    for i in range(len(node_list) - 1):
        q.add_edge(node_list[i], node_list[i + 1])
    
    find_belief(q)

In [16]:
stress_belief_mini()

For Node 11
Belief, T: 2.2564410783484135e-05
Belief, F: 1.2352060995496844e-05
Eventual belief: T 

For Node 12
Belief, T: 0.002858431348649789
Belief, F: 0.0016721823367389925
Eventual belief: T 

For Node 21
Belief, T: 0.017079040870927778
Belief, F: 0.006273381502871841
Eventual belief: T 

For Node 22
Belief, T: 0.00010781552025765446
Belief, F: 1.3235081836888983e-07
Eventual belief: T 



In [17]:
stress_belief()

For Node 22
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node 23
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node 24
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node 25
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node 26
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node 27
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node 28
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node 29
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node 2:
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node 2;
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node 2<
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node 2=
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node 2>
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node 2?
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node 2@
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node 2A
Belief, T: 0.0
Belief, F: 0.0
Eventual beli

Belief, F: 0.0
Eventual belief: F 

For Node 5>
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node 5?
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node 5@
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node 5A
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node 5B
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node 5C
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node 5D
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node 5E
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node 5F
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node 5G
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node 5H
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node 5I
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node 5J
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node 5K
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node 5L
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node 5M
Belief,

Eventual belief: F 

For Node 9T
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node 9U
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node 9V
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node 9W
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node 9X
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node 9Y
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node 9Z
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node :2
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node :3
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node :4
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node :5
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node :6
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node :7
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node :8
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node :9
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node ::
Belief, T: 0.0
Belief,

Eventual belief: F 

For Node >R
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node >S
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node >T
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node >U
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node >V
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node >W
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node >X
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node >Y
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node >Z
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node ?2
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node ?3
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node ?4
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node ?5
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node ?6
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node ?7
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node ?8
Belief, T: 0.0
Belief,

Eventual belief: F 

For Node B;
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node B<
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node B=
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node B>
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node B?
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node B@
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node BA
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node BB
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node BC
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node BD
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node BE
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node BF
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node BG
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node BH
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node BI
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node BJ
Belief, T: 0.0
Belief,

...

For Node IF
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node IG
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node IH
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node II
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node IJ
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node IK
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node IL
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node IM
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node IN
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node IO
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node IP
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node IQ
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node IR
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node IS
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node IT
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node IU
Belief, T: 0.0
Belief, F: 0.0
Eventual beli

For Node LS
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node LT
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node LU
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node LV
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node LW
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node LX
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node LY
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node LZ
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node M2
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node M3
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node M4
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node M5
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node M6
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node M7
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node M8
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node M9
Belief, T: 0.0
Belief, F: 0.0
Eventual beli

For Node P3
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node P4
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node P5
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node P6
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node P7
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node P8
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node P9
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node P:
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node P;
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node P<
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node P=
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node P>
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node P?
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node P@
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node PA
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node PB
Belief, T: 0.0
Belief, F: 0.0
Eventual beli

Eventual belief: F 

For Node ST
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node SU
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node SV
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node SW
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node SX
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node SY
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node SZ
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node T2
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node T3
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node T4
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node T5
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node T6
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node T7
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node T8
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node T9
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node T:
Belief, T: 0.0
Belief,

For Node WH
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node WI
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node WJ
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node WK
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node WL
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node WM
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node WN
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node WO
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node WP
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node WQ
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node WR
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node WS
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node WT
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node WU
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node WV
Belief, T: 0.0
Belief, F: 0.0
Eventual belief: F 

For Node WW
Belief, T: 0.0
Belief, F: 0.0
Eventual beli

As you can see, the algorithm evaluates almost immediately (albeit giving a bunch of zeros) because there are so many nodes. This can be shown from the **stress_belief_mini** where the belief values are actually very low. At any rate, the stress stressproves that our algorithm is efficient and follows the dynamic programming paradigm. We can now move on to the next question on Huffman Code.

### Solution for Problem 3 - Huffman Code

**(a) Write python code to implement the Huffman coding to compress any given strings. Specifically, given an input of sentences stored in a possibly long array, your final output is the binary digits representing the input sentences in an array. Your second output is the binary digit representation of every character in the input array.**

In this question, we have to implement the Huffman Code algorithm to efficiently compress input sentences in a lossless manner. Before starting on this question, it would be good to define some fundamental data structures first. We first define the class **HuffmanNode** first. this is the Class that is described in the lecture notes as the Huffman Tree with a left and right node. However, we have another class **self.table**. This is for the root node to store the Huffman Code for the Tree. 

In the lecture notes, it is mentioned that the HuffmanNode is done using a Minimum Priority Queue. I choose to implement this using a Min-Heap data structure explained in Chapter 6. Other than that, there isn't anything really special. The only catch here is that in the zero-th index of this heap, I create a Huffman Node with value **-sys.maxsize**(multiplied by -1). This makes it easier to refer to the indexes of the array without pesky plus/minus 1s to the index. Other than that, the Minimum Priority Queue is done in terms of the frequency of which the characters appear in a sentence. 

The implementation of these 2 basic structures are shown below:

In [18]:
import sys

class HuffmanNode:
    def __init__(self, freq, char = None, left = None, right = None):
        """
        Huffman Tree with a Left and Right Node. 
        self.table is used to store Huffman Code,
        self.freq is used to store frequency at
        which the character appears. 
        """
        self.left = left
        self.right = right
        self.char = char
        self.freq = freq
        self.table = {}
        
class MinPriorityQueue:
    def __init__(self, capacity):
        self.limit = capacity 
        self.size = 0
        self.Heap = [None] * (capacity + 1)
        self.Heap[0] = HuffmanNode(sys.maxsize * -1)
        self.root = 1

    
    def swap(self, pos1, pos2):
        """
        Swap operation to switch array indexes. Used to maintain min-heap
        property later on in min_heapify. 
        """
        self.Heap[pos1], self.Heap[pos2] = self.Heap[pos2], self.Heap[pos1]
        
    def parent(self, pos):
        """
        Returns index of parent node
        """
        return pos // 2
    
    def left(self, pos):
        """
        Returns index of left child
        """
        return pos * 2
    
    def right(self, pos):
        """
        Returns index of right child
        """
        return pos * 2 + 1
    
    def min_heapify(self, pos):
        """
        Auxillary function to maintain min-heap property
        """
        l = self.left(pos)
        r = self.right(pos)
        if l <= self.size and self.Heap[l].freq < self.Heap[pos].freq:
            low = l
        else:
            low = pos
        if r <= self.size and self.Heap[r].freq < self.Heap[low].freq:
            low = r
        if low != pos:
            self.swap(pos, low)
            self.min_heapify(low)
        
    def build_minheap(self):
        """
        Build min heap from array
        """
        for i in range(self.size // 2, 0, -1):
            self.min_heapify(i)
            
    def heap_min(self):
        return self.Heap[1]
            
    def extract_min(self):
        """
        Auxillary function to extract minimum 
        element in the array. 
        """
        if self.size < 1:
            raise ValueError("Heap Underflow")
        low = self.Heap[1]
        self.Heap[1] = self.Heap[self.size]
        self.Heap[self.size] = None
        self.size -= 1
        self.min_heapify(1)
        return low
    
    def decrease_key(self,i, key):
        if key.freq > self.Heap[i].freq:
            raise ValueError("New key Larger than Current key")
        self.Heap[i] = key
        while i > 1 and self.Heap[self.parent(i)].freq > self.Heap[i].freq:
            self.swap(self.parent(i), i)
            i = self.parent(i)
            
    def insert(self,key):
        """
        Inserts key to the Queue
        """
        self.size += 1
        self.Heap[self.size] = HuffmanNode(sys.maxsize)
        self.decrease_key(self.size, key)

Now that we are done with this, we move next to the implementation of the algorithm. We have the main implementation of the huffman code. While this is adapted greatly from the lecture note, there are some auxillary functions that I have defined to help me in this execution.

Firstly, An array of sentences is passed into the function **huffman_code**. Afterwards, it scans each sentence of the array and then scans each character of the array. the variable **code** acts as a frequency table to track how many instances of the character was there.

Secondly, a Minimum Priority Queue is created with length that matches the number of unique characters in the frequency table. Afterwards, all of these character-value pairs in the dictionary are converted to singleton Huffman Nodes. Afterwards, the Huffman Algorithm runs (as per the lecture) note for **(n - 1)** times, where **n** is the number of unique characters as shown in the frequency table. At the very end, this should have created a huge Huffman Tree. 

Next, this huge Huffman Tree is passed unto the function **assign_code**, which then creates a huffman code for each character that is stored at the root node of the Huffman Tree. We extract this out and name the variable **code** and as usual, traversing to the left add a "0" while traversing to the right in the recursive function adds a "1" to the final code. 

Last but out last, we have a final function **encode** which encodes the sentence with the corresponding code and returns it according to the question requirements. 

In [19]:
def assign_code(node, root, seq = ""):
    if type(node.char) == str:
        root.table[node.char] = seq                
    else:                              
        assign_code(node.left, root, seq + "0")    
        assign_code(node.right, root, seq + "1")
    
def encode(sentence, code):
    """
    sentence - string of sentence 
    code - Huffman Code
    
    Encode takes in a sentence and 
    converts it to a string of binary
    digits representing the sentence
    """
    out = ""
    
    for char in sentence:
        out = out + code[char]
    
    return out
    
def huffman_code(arr):
    """
    arr - array of sentences
    
    Main function of generating the
    sentences and the binary strings
    representing the sentence. 
    """ 
    seen = {}
    for sentence in arr:
        for c in sentence:
            if c not in seen:
                seen[c] = 1
            else: 
                seen[c] += 1
                
    # base case: no sentences, frequency table is empty
    if seen == {}:
        print("No sentences. Return empty array and empty code")
        return [], {}

    q = MinPriorityQueue(len(seen.keys()))

    for char, freq in seen.items():
        q.insert(HuffmanNode(freq, char))
        
    for i in range(len(seen.keys()) - 1):
        x = q.extract_min()
        y = q.extract_min()
        z = HuffmanNode(x.freq + y.freq, None, x, y)
        q.insert(z)

    huffman = q.extract_min()
    assign_code(huffman, huffman)
    code = huffman.table
    
    out = []
    for sentence in arr:
        out.append(encode(sentence, code))
    
    # print(seen)
    # print(out)
    # print(code)
    return out, code

Here are some basic tests for the Huffman Code: one for the empty list case and empty string case and on a small input string. As we can see, the huffman performs normally. It gives small code for frequently appearing characters like 's' and '9' while giving long code for seldomly appearing characters like '3' and '1'.

In [20]:
print("Empty test")
huffman_code([""])
huffman_code([])

print("test on small array")
huffman_code(["999999999ssssssss888772", "ssssss77213"])

Empty test
No sentences. Return empty array and empty code
No sentences. Return empty array and empty code
test on small array


(['1010101010101010100000000011101110111011011011110',
  '00000011011011110111110111111'],
 {'s': '0',
  '9': '10',
  '7': '110',
  '8': '1110',
  '2': '11110',
  '1': '111110',
  '3': '111111'})

Last but not least, there is a stress test for longer sentences to prove that it runs properly. That being said, the Huffman Code works better than there is a character with a very large frequency, and not a sentence where the character frequency is roughly homogenous. However, this does prove that our implementation works for larger sentences. Run this line if you'd like to see the output proper.

In [21]:
import random
import string

def stress_huffman():
    arr = []
    n_sentences = random.randint(10, 20)
    letters = string.ascii_lowercase
    for i in range(n_sentences):
        length = random.randint(50, 100)
        s = ''.join(random.choice(letters) for i in range(length))
        arr.append(s)
    
    encoded_arr, code = huffman_code(arr)
    return encoded_arr, code

stress_huffman()

(['11011110110111001111110001110001011000001111010110110100100011011101110100110001100101010011000111101011000101110110010101111001100001010011110101111010010000101111100011011010110101111110010100001101110001001011000001010100010101010000101101011111001100111010000000010100011100001000111101001001010111100111101111101110011101111000111111110111000111011001010010110111110100110011100000010000010000',
  '0001001011100110010111000110110110010100001101001001100100111101001100010100010001011001111001101001100011001010111010011111000010100010111000001111010000010111000011000111011010000010001110011010000010011111111111100110101011001010110000',
  '0101010011011010101110110100100111111111110001000010111111101001010110101001010011010010011000010110111110111011010001011001101001101100011111001111010111101000001101000000101101111111010110110000111110101011001100101011010110111010000110101100010111100001100010101100111001110100110010111110011111011010100000010110101011101110100010101001000101000

**(c) Argue that your tree representation is stored efficiently in terms of memory allocation. Hint:
the tree allocation must be dynamic.**

In order to answer this question, we need to make reference to the Min Heap. Python doesn't really come with capabilities to garbage collect and do all the cool memory allocation stuff. However, after dequeuing the Node, we can set the corresponding index to None after it's done. That way, we save memory. 

The intuition here is that as the dequeuing happens, more data is freed from the array. As we do the dequeuing, the tree gets larger until eventually we find out that the queue is empty, with the exception of the dummy node that contains **-sys.maxsize**. The below code snippet shows how a traced version of **huffman_code** without returning the huffman code and sentence, but concentrates on the queue after enqueuing and dequeuing. This implementation is efficient in terms of memory as the size of the queue decreases - there are more and more **None** values as the dequeuing proceeds.

In [22]:
def huffman_code_traced(arr):
    """
    arr - array of sentences
    """
    seen = {}
    for sentence in arr:
        for c in sentence:
            if c not in seen:
                seen[c] = 1
            else: 
                seen[c] += 1
                
    # base case: no sentences, frequency table is empty
    if seen == {}:
        print("No sentences. Return empty array and empty code")
        return [], {}

    q = MinPriorityQueue(len(seen.keys()))
    print("Initial queue: ")
    print(q.Heap)
    
    for char, freq in seen.items():
        q.insert(HuffmanNode(freq, char))
    
    print("\n\nQueue after completely enqueuing all the Huffman Nodes: ")
    print(q.Heap)
    
    for i in range(len(seen.keys()) - 1):
        x = q.extract_min()
        y = q.extract_min()
        z = HuffmanNode(x.freq + y.freq, None, x, y)
        q.insert(z)

    huffman = q.extract_min()
    
    print("\n\nQueue after complete dequeuing of Huffman Nodes: ")
    print(q.Heap)
    assign_code(huffman, huffman)
    code = huffman.table
    
    out = []
    for sentence in arr:
        out.append(encode(sentence, code))
    
    return 

print("testing on small array")
huffman_code_traced(["999999999ssssssss888772", "ssssss77213"])

testing on small array
Initial queue: 
[<__main__.HuffmanNode object at 0x00000284596AB948>, None, None, None, None, None, None, None]


Queue after completely enqueuing all the Huffman Nodes: 
[<__main__.HuffmanNode object at 0x00000284596AB948>, <__main__.HuffmanNode object at 0x00000284596AAB48>, <__main__.HuffmanNode object at 0x00000284596AA6C8>, <__main__.HuffmanNode object at 0x00000284596AAE48>, <__main__.HuffmanNode object at 0x00000284596AABC8>, <__main__.HuffmanNode object at 0x00000284596AAB08>, <__main__.HuffmanNode object at 0x00000284596AB148>, <__main__.HuffmanNode object at 0x00000284596AA148>]


Queue after complete dequeuing of Huffman Nodes: 
[<__main__.HuffmanNode object at 0x00000284596AB948>, None, None, None, None, None, None, None]


### Solution for Problem 4 - Maximum Flow/ Minimum Cut 

**(a) Any input of the code in Question 2 can be the input of this code.**

In this particular case, I made an inherited class called **DirectedGraph** which inherits all the methods of the previous problem 2. This problem essentially adds to the **Graph** functionality of the question, preserves it and redefines it in a way that makes solving this question viable. In other words, when solving the Ford Fulkerson Algorithm, one should use **add_edge** and **add_node** from the original **Graph** class until feeding this input into the Ford Fulkerson Algorithm, which converts it to a Graph. 

The class **DirectedGraph** has a few additional properties:
 - Firstly, I have additionally defined a Source Node. The Source Node is connected to all vertices which have been defined before calling this class. The Edge value will take on the "TRUE"(Node.T) value of the original node.
 - Secondly, I have additionally defined a Sink Node. The Sink Node is connected to all vertices which have been defined before calling this class. The Edge value will take on the "False"(Node.F) value of the original node.
 - For the previously undirected edges between two nodes, they are directed edges that go both ways in the Graph (e.g. a-b becomes a->b and b->a in the new graph). The weight between them has been assigned to the AND value of the influence previously defined in Question 2.
 
When calling the **DirectedGraph** class, this class will be created from the old **Graph** class. Afterwards, the **setup** function will be performed, essentially performing all the operations that I have done just now. Below shows the specifics of my implementation and also basic tests on this class which also serves as a mini tutorial on how to use it:

In [23]:
class DirectedGraph(Graph):
    def __init__(self, G):
        """
        Initializing the DirectedGraph class.
        It requires an input G which is of
        Graph class defined in Problem 2.
        """
        self.undirected = G
        self.graph = {}
        self.graph["Source"] = {}
        self.graph["Sink"] = {}
        self.setup()
    
    def setup(self):
        """
        Setup function to create the Directed Graph
        Necessary to execute the Ford Fulkerson
        Algorithm. Autorun on initialization.
        """
        for label, node in self.undirected.G.items():
            self.graph["Source"][label] = node.T       # edge from source to vertice
            self.graph[label] = {}                     # Because this has not been initialized yet
            self.graph[label]["Sink"] = node.F         # edge from non-source vertice to sink
            for (key, adjacent) in node.link.items():  
                self.graph[label][key] = adjacent.AND  # between 2 nodes in the graph, using AND value
    
    def display(self):
        """
        Pretty print function that displays all 
        vertices, connections and its weights.
        """
        for key, neighbour in self.graph.items():
            print("\nCurrently visiting node", key)
            for vertice, val in neighbour.items():
                print("Adjacent to:", vertice + ". Value:", val)

# basic illustration and testing functions

g = Graph()
g.add_node("a")
g.add_node("b")
g.add_edge("a", "b")

g = DirectedGraph(g)
g.display() # no nodes from sink to elsewhere by definition


Currently visiting node Source
Adjacent to: a. Value: 0.5238256182203845
Adjacent to: b. Value: 0.10582653247911833

Currently visiting node Sink

Currently visiting node a
Adjacent to: Sink. Value: 0.47617438177961546
Adjacent to: b. Value: 0.462459429655966

Currently visiting node b
Adjacent to: Sink. Value: 0.8941734675208817
Adjacent to: a. Value: 0.462459429655966


The below shows the algorithm that I have used for the Ford-Fulkerson Algorithm. There are a number of functions that I have used in this question. Allow me to expand on all of them: 

**bfs_path** - This is a breadth-first search function that helps me to find out a possible path in a greedy manner. In order to prevent said function from encountering a cycle (loop indefinitely), measures have been put in place to detect when a node (neighbour has already been visited) 

**extract_path** - Essentially, this function takes the path dictionary and backtracks a path back to the source node. This is very useful as the next function helps to find the min feasible flow along that path, which helps to augment the graph proper. This is returned as a list, ordered from the source traversing all the way to the sink. 

**find_flow** - This function takes in 2 arguments: The graph (to be augmented) and also the path that is found. It corresponds the edge to the graph in a for loop through the path, trying to find the minimum flow. and returning its value and passing it to the next function - augment_graph which helps to create the reverse flow 

**augment_graph** - This function essentially augments the graph. It does numerous things:
    - Firstly, it accepts a flow, path corresponding to that flow, forward residual graph and augmented graph (containing both forward and backward residual edges)
    - Based on the path and flow, it creates the forward residual graph by subtracting the capacity by the flow. This is done to the forward residual graph (residual)
    - The previous operation is also done to the augmented graph. However, if the corresponding edge of forward residual graph has a capacity of 0, then this edge is disconnected from the graph since it is already saturated. In addition, the backward residual graph is also created in the augmented graph in accordance of the Ford-Fulkerson algorithm.

**source_set** - Given a forward residual graph, this function finds all the vertices (except sink) that are accessible from the source. In the very end, the nodes within the source set will be given the TRUE ("T") label, but the nodes that are not included in this set are given the FALSE("F") label.

**Fold_Fulkerson** - The main algorithm driving this function together. In essence, this function combines all the ideas of the previous functions and tries to find a max-flow/min-cut answer and returns it. I have included a min-cut function for fun.

In [24]:
import sys

def bfs_path(G):
    """
    Utility Function of BFS. Employs a greedy
    Strategy to try and find a path from source
    node to sink node that has non-zero flow.
    """
    queue = []
    queue.append("Source")
    out = {}
    while queue:
        # get first path from queue
        node_label = queue.pop(0)
        for neighbour in G.graph[node_label]: 
            if neighbour in out.values(): # if neighbour has already been visited
                continue
            out[neighbour] = node_label   # the edge direction goes from neighbour <- node_label 
            if neighbour == "Sink":       # we have already bound a path from source to sink
                return out
            else: 
                queue.append(neighbour)   # appending this to queue in BFS style
    
    return None
        
def extract_path(path_dict):
    """
    Extract path takes the returned bfs path
    expressed as a dictionary and then backtracks 
    the path from source node to sink node, 
    constructing the path as a List from source
    to sink.
    """
    if path_dict is None:
        return None
    
    out = []
    curr = 'Sink'
    while curr != 'Source':
        out.insert(0, curr)
        curr = path_dict[curr]
    
    out.insert(0, 'Source')
    return out

def find_flow(path, G):
    """
    This function takes the extracted path and
    finds the corresponding minimum flow in the graph
    """
    min_flow = sys.maxsize
    for i in range(len(path) - 1):
        begin = path[i]
        end = path[i + 1]
        min_flow = min(G.graph[begin][end], min_flow)
    return min_flow

def augment_graph(flow, G, residual, path):
    """
    flow - min flow
    G - graph to be augmented, containing forward/ backward residual graphs
    residual - forward residual graph
    path - path found by BFS 
    
    Essentially, this function performs the Ford-Fulkerson augmentation.
    However, I am cloning the forward residual graph as I can use that 
    to find the source set - vertices that the source node can reach.
    """
    for i in range(len(path) - 1):
        begin = path[i]
        end = path[i + 1]
        G.graph[begin][end] -= flow
        residual.graph[begin][end] -= flow
        if begin not in G.graph[end]:
            G.graph[end][begin] = flow
        else: 
            G.graph[end][begin] += flow
        
        if G.graph[begin][end] == 0:
            del G.graph[begin][end]
    
        
def source_set(residual):
    """
    Given a residual graph from the capacities, this function 
    gives the source set: vertices that are reachable from
    the source set (non-zero flow). This is important as it
    helps us calculate the 
    """
    source_set = set()
    source_set.add("Source")
    queue = ["Source"]
    while queue:
        node_label = queue.pop(0)
        
        for neighbour in residual.graph[node_label]:
            if neighbour == "Sink" and residual.graph[node_label][neighbour] != 0:
                raise ValueError("You're screwed. How is there a path to sink!?")
                
            if residual.graph[node_label][neighbour] != 0 and neighbour not in source_set:
                source_set.add(neighbour)
                queue.append(neighbour)
            
    return source_set
            
            
def Ford_Fulkerson(G):
    """
    Main function to find the max-flow/min-cut cost,
    together with the optimal labels of the nodes. 
    """
    capacity = DirectedGraph(G)
    residual = DirectedGraph(G)
    G = DirectedGraph(G)
    temp = bfs_path(G)
    path = extract_path(temp)
    # this function greedily finds a path until there
    # are no more viable parts 
    while path: 
        min_flow = find_flow(path, G)
        augment_graph(min_flow, G, residual, path)
        temp = bfs_path(G)
        path = extract_path(temp)

    max_flow = 0
    
    # max_flow calculated from the sum of the inflows 
    # to the sink node 
    for value in G.graph["Sink"].values():
        max_flow += value
        
    s_set = source_set(residual)
    tag = {}
    # finalized optimized labels of the source set 
    for labels in G.graph:
        if labels == "Source" or labels == "Sink":
            continue
        else:
            if labels in s_set:
                tag[labels] = "T"
            else:
                tag[labels] = "F"
    
    # bonus to find min-cut
    min_cut = 0
    for label in s_set:
        for neighbour, flow in capacity.graph[label].items():
            if neighbour not in s_set:
                min_cut += flow
            
    print("Min-Cut is", min_cut)
    print("Max-Flow is", max_flow)
    print("They're definitely equal.")
    print("Finalized labels are:", tag)
    return max_flow, tag

Now that we are done constructing the algorithm, we can now run some tests to satisfy the remaining parts: 

**(a) Any input of the code in Question 2 can be the input of this code.**

As you can see in my inputs to the Ford-Fulkerson algorithm, I am inputting an argument belonging to the **Graph** class. Therefore, I satisfy this requirement.

**(b) The output should be the same as the output of the code in Question 2, namely the optimized
label of each node.**

In each instance of the execution, I have also printed the optimized label of each node - whether it should be True (Source) or False (Sink). I have also returned this as a variable in my function. 

**(c) Graph traversal must be depth first.**

As spoken in class, the Graph traversal need not be DFS, but BFS is also okay. Since I felt that doing this was easier, I went for it. 

**(d) Cost of max-flow/min-cut must be shown.**

Just like part (c), the cost of max-flow/min-cut has been shown as a print message and as a return variable

**(e) Unlike Question 2, this code must be able to handle cyclic undirected graphs.**

An example of this has been shown in the bottommost code snippet, where now, my

**(f) Provide at least one example of the input and the output.**

Below, I have provided 2 examples of the input and output. One of was was exactly what you went through in class, and the results match exactly. I have provided an example of a cyclic undirected graph in the second code snippet. 

In [25]:
g = Graph()
g.add_node("1", 0.7)
g.add_node("2", 0.4)
g.add_edge("1", "2", 0.1)
Ford_Fulkerson(g)

Min-Cut is 0.8
Max-Flow is 0.8
They're definitely equal.
Finalized labels are: {'1': 'T', '2': 'F'}


(0.8, {'1': 'T', '2': 'F'})

In [26]:
g = Graph()
g.add_node("1", 0.2)
g.add_node("2", 0.7)
g.add_node("3", 0.8)
g.add_node("4", 0.1)
g.add_edge("1", "2", 0.2)
g.add_edge("2", "3", 0.1)
g.add_edge("3", "4", 0.2)
g.add_edge("1", "4", 0.1)
Ford_Fulkerson(g)

Min-Cut is 1.2
Max-Flow is 1.2000000000000002
They're definitely equal.
Finalized labels are: {'1': 'F', '2': 'T', '3': 'T', '4': 'F'}


(1.2000000000000002, {'1': 'F', '2': 'T', '3': 'T', '4': 'F'})